# Optimisation of commerical building energy system

Model of a large commercial building (100kW mean load is pretty hefty)

Setup stuff

In [1]:
from tqdm.auto import tqdm
import numpy as np
from numpy import random
from scipy import stats
from models import run_model
from functools import partial
from utils import get_Gurobi_WLS_env, fmt_design_results

In [2]:
env = get_Gurobi_WLS_env(silence=True)
run_model = partial(run_model, env=env)

In [3]:
solar_years = range(2012,2018)
load_years = range(2012,2018)
random.seed(42)

In [4]:
design_options = [
    [500,200],
    [500,400],
    [600,500],
    [750,600],
    [750,800]
]

In [5]:
results = [run_model(*design) for design in tqdm(design_options, desc='Assessing designs')]

Assessing designs:   0%|          | 0/5 [00:00<?, ?it/s]

In [40]:
best_design = results[np.argmin([r['total'] for r in results])]
print(fmt_design_results(best_design))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       750
Battery capacity  kWh       600
Total cost        £k/yr     168.4
CAPEX             £k/yr     102
OPEX              £k/yr      66.4


In [41]:
nosystem_results = run_model(0, 0)
print(fmt_design_results(nosystem_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp         0
Battery capacity  kWh         0
Total cost        £k/yr     214.2
CAPEX             £k/yr       0
OPEX              £k/yr     214.2


Therefore, installing the solar-battery system saves ..., i.e. reduces total cost of providing building load by ...%

#### Designing using the Linear Program

In [42]:
LP_results = run_model(None,None)
print(fmt_design_results(LP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       834.6
Battery capacity  kWh       343.3
Total cost        £k/yr     161.7
CAPEX             £k/yr      90.8
OPEX              £k/yr      70.9


Improvement from LP vs static options

In [43]:
print(f"£{round((best_design['total'] - LP_results['total'])/1e3,2)}k/yr")
print(f"{round((best_design['total'] - LP_results['total'])/best_design['total']*100,2)}%")

£6.71k/yr
3.99%


#### Accounting for uncertainty during design

In [45]:
avg_costs = []
for design in tqdm(design_options):
    avg_cost = np.mean([run_model(*design, solar_year=[year])['total'] for year in tqdm(solar_years,leave=False)])
    avg_costs.append(avg_cost)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [46]:
print(min(avg_costs))
print(design_options[np.argmin(avg_costs)])

166619.95467501873
[750, 600]


In [47]:
r=run_model(750,600,solar_year=solar_years)

In [49]:
print(r['total'])

166619.9546750187


Could use this little trick of getting the solver to optimise independent scenarios simultaneously - looks like it's giving the right numbers

Look at cost estimation error of deterministic optimisation

In [11]:
print(min(avg_costs) - min(costs))
print((min(avg_costs) - min(costs))/min(avg_costs) * 100)

-1776.4531883149757
-1.0661707307386026


SP ...

In [14]:
SP_results = run_model(None,None,solar_year=list(range(2012,2018)))
print(fmt_design_results(SP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       794.8
Battery capacity  kWh       340.7
Total cost        £k/yr     160.5
CAPEX             £k/yr      87.4
OPEX              £k/yr      73.1


Improvement from SP vs static options

In [15]:
print((SP_results['total'] - min(avg_costs))/min(avg_costs)*100)

-3.673709122439898


Stochastic optimisation with lots of uncertainties ...

In [16]:
nsamples = 10
scenarios = {
    'solar_year': random.choice(solar_years, nsamples),
    'load_year': random.choice(load_years, nsamples),
    'mean_load': stats.truncnorm.rvs(-2, 2, loc=100, scale=10, size=nsamples),
    'battery_efficiency': stats.truncnorm.rvs(-2, 2, loc=0.95, scale=0.05, size=nsamples),
    'battery_cost': stats.truncnorm.rvs(-2, 2, loc=70, scale=5, size=nsamples),
}

In [17]:
avg_costs = []
for design in tqdm(design_options):
    avg_cost = np.mean([run_model(
        *design,
        solar_year=[scenarios['solar_year'][i]],
        load_year=[scenarios['load_year'][i]],
        mean_load=[scenarios['mean_load'][i]],
        battery_efficiency=[scenarios['battery_efficiency'][i]],
        battery_cost=[scenarios['battery_cost'][i]]
        )['total']
            for i in tqdm(range(nsamples),leave=False)])
    avg_costs.append(avg_cost)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
print(min(avg_costs))
print(design_options[np.argmin(avg_costs)])

159855.46900462912
[750, 600]


In [19]:
SP_results = run_model(
    solar_capacity=None,
    battery_capacity=None,
    solar_year=scenarios['solar_year'],
    load_year=scenarios['load_year'],
    mean_load=scenarios['mean_load'],
    battery_efficiency=scenarios['battery_efficiency'],
    battery_cost=scenarios['battery_cost']
)

In [20]:
print(fmt_design_results(SP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       772.9
Battery capacity  kWh       400.3
Total cost        £k/yr     156.6
CAPEX             £k/yr      90.1
OPEX              £k/yr      66.4


Could also then look at benefit of continuous solution (i.e. SP vs discrete stochastic), and Value of Stochastic Solution